In [1]:
import requests#Connect to endpoints
import time #time script
import sys
import csv
import pandas as pd
import spotipy#authentication
import spotipy.util as util#authentication
from spotipy.oauth2 import SpotifyClientCredentials#authentication

In [2]:
#Credentials
cid = '049ade7215e54c63a2b628f3784dc407'
secret = 'b86bedbebf53486b970e92b2cb7b612c'
redirect_uri = 'http://google.com/'
username = 'xxx'

In [3]:
#Authentication
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
def get_artist(url):
    
    #Request url
    try:
        resp = requests.get(url=url,
                            headers={'Authorization': 'Bearer ' + token})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        return None
        
    #Response in json
    response = resp.json()

    #Get info from json keys
    try:
        artist_uri = response['artists']['items'][0]['uri'].split(':')
        artist_name = response['artists']['items'][0]['name'].replace('[','').replace(']','')
        artist_name = artist_name.strip()
        followers = response['artists']['items'][0]['followers']['total']
        
        #Create dictionary of artist info
        artist = {'Artist URI':artist_uri[2],'Artist Name':artist_name,'Followers':followers}
        return artist
    
    except IndexError as error:
        return
    
    

In [9]:
i = 0
artist_list=[]

start = time.time()
with open('Data/artists_final.csv') as file:
    next(file)
    head = [next(file) for x in range(25)] #for limiting amount of rows read
    for row in head:
        #with open('Data/Spotify_Musicdb_artist_id.txt','a') as outfile:
            artist_name = row.strip('\n').strip()
            artist_search = artist_name.replace(' ','%')
            artist_info = get_artist('https://api.spotify.com/v1/search?q={a}&type=artist&limit=1'
                                     .format(a=artist_search))#output artist uri
            print(i,artist_info)#Get artist 
            i+=1
            if artist_info is not None: #Some artists in msdb don't exist in Spotify
                artist_list.append(artist_info)
            else:
                with open('Kickout/kickout_artist.csv', 'w+', newline='\n') as file:
                    file.write('{a}|{s}\n'.format(a=artist_name,s=artist_search))
stop = time.time()
duration = (stop - start)/60

0 {'Artist URI': '3bBWKHfpepPOychRNFzg4q', 'Artist Name': 'Al Di Meola', 'Followers': 79843}
1 {'Artist URI': '3bBWKHfpepPOychRNFzg4q', 'Artist Name': 'Al Di Meola', 'Followers': 79843}
2 {'Artist URI': '3h8OjAdgYXVRoMZ8jFd6Uw', 'Artist Name': 'Paco de Lucía', 'Followers': 313314}
3 {'Artist URI': '3h8OjAdgYXVRoMZ8jFd6Uw', 'Artist Name': 'Paco de Lucía', 'Followers': 313314}
4 None
5 {'Artist URI': '3bBWKHfpepPOychRNFzg4q', 'Artist Name': 'Al Di Meola', 'Followers': 79843}
6 {'Artist URI': '3bBWKHfpepPOychRNFzg4q', 'Artist Name': 'Al Di Meola', 'Followers': 79843}
7 {'Artist URI': '2Z8XcWdgy5dSaiQR3d7tHK', 'Artist Name': 'Armik', 'Followers': 20444}
8 {'Artist URI': '3UvcmAOZt64oKpP95f6MMM', 'Artist Name': 'Big Star', 'Followers': 89435}
9 None
10 {'Artist URI': '2gCsNOpiBaMNh20jQ5prf0', 'Artist Name': 'Buddy Guy', 'Followers': 558811}
11 {'Artist URI': '6paRdA1dkAEb0dWcIS4TOp', 'Artist Name': 'Buffalo Tom', 'Followers': 25046}
12 {'Artist URI': '2lMw9KHEs2ymoFBRuSrMHk', 'Artist Name':

In [6]:
#Duration of script
str(duration)+' minutes'

'0.06192180315653483 minutes'

In [8]:
#Display
df = pd.DataFrame(artist_list)
#df
#There are duplicate artist because for a multiple artist search, it returns the first artist
no_duplicates = df.drop_duplicates(['Artist URI'])
no_duplicates
#no_duplicates.to_csv('Data/artist_uri.csv',sep='|',index=False)

,Artist URI,Artist Name,Followers
0,3bBWKHfpepPOychRNFzg4q,Al Di Meola,79843
2,3h8OjAdgYXVRoMZ8jFd6Uw,Paco de Lucía,313314
6,2Z8XcWdgy5dSaiQR3d7tHK,Armik,20444
7,3UvcmAOZt64oKpP95f6MMM,Big Star,89435
8,2gCsNOpiBaMNh20jQ5prf0,Buddy Guy,558811
9,6paRdA1dkAEb0dWcIS4TOp,Buffalo Tom,25046
10,2lMw9KHEs2ymoFBRuSrMHk,Caitlin Cary,1033
11,3mv2eHwrIONSkviASDMRMa,Califone,10734
12,4Bgfzia8wyaGaTn42egzTv,Catatonia,37619
13,2XJzOLYV2mF5K2JfUhJEK0,Chris Knight,52095
